In [1]:
import sys
!{sys.executable} -m pip install pandas openpyxl

  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.1.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.0 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [2]:
import os
import shutil
import pandas as pd

In [3]:

def flatten_directories(main_folder1, main_folder2, target_folder):
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    def move_files_from_folder(folder):
        for subdir, _, files in os.walk(folder):
            for file in files:
                # Construct full file path
                file_path = os.path.join(subdir, file)
                
                # Construct the target file path
                target_file_path = os.path.join(target_folder, file)
                
                # Handle file name conflicts
                if os.path.exists(target_file_path):
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_file_name = f"{base}_{counter}{extension}"
                    new_target_file_path = os.path.join(target_folder, new_file_name)
                    
                    while os.path.exists(new_target_file_path):
                        counter += 1
                        new_file_name = f"{base}_{counter}{extension}"
                        new_target_file_path = os.path.join(target_folder, new_file_name)
                    
                    target_file_path = new_target_file_path
                
                # Move the file to the target directory
                shutil.move(file_path, target_file_path)
    
    # Move files from both main folders
    move_files_from_folder(main_folder1)
    move_files_from_folder(main_folder2)
    
    print("All files have been moved successfully.")

# Example usage
main_folder1 = '../data/raw/batch1'
main_folder2 = '../data/raw/batch2'
target_folder = '../data/raw/flat'

flatten_directories(main_folder1, main_folder2, target_folder)



All files have been moved successfully.


In [4]:
def process_excel_files(folder_path, output_file_name):
    # List to store individual DataFrames
    all_dataframes = []

    # Construct the full path for the 'flat' folder containing the raw data
    folder_path_to_raw_data = os.path.join(folder_path, 'raw/flat')

    # Loop through all files in the folder
    for filename in os.listdir(folder_path_to_raw_data):
        # Ignore hidden files such as .DS_Store in MacOS
        if not filename.startswith('.'): 
            file_path = os.path.join(folder_path_to_raw_data, filename)

            # Load the metadata rows
            metadata_df = pd.read_excel(file_path, header=None, nrows=8)
            metadata_df = metadata_df[[0, 3]]
            new_column_names = metadata_df.iloc[:, 0].tolist()
            new_column_values = metadata_df.iloc[:, 1].tolist()

            # Load the main data (assuming the data starts after the metadata rows)
            data_df = pd.read_excel(file_path, skiprows=8)
            
            # Add new columns to the dataframe
            for i, column_name in enumerate(new_column_names):
                data_df[column_name] = new_column_values[i]

            # Remove rows without a row ID (assuming 'ID' is the first column after the header)
            data_df = data_df.dropna(subset=[data_df.columns[0]])

            # Append the processed DataFrame to the list
            all_dataframes.append(data_df)

    # Concatenate all the DataFrames in the list
    appended_df = pd.concat(all_dataframes, ignore_index=True)

    # Determine the parent directory of the specified folder
    parent_dir = os.path.abspath(os.path.join(folder_path, 'processed'))
    
    # Construct the full path for the output file in the parent directory
    output_file_path = os.path.join(parent_dir, output_file_name)

    # Save the concatenated DataFrame to an Excel file in the parent directory
    appended_df.to_excel(output_file_path, index=False)

# Example usage
folder_path = '../data'
output_file_name = 'appended_data.xlsx'
process_excel_files(folder_path, output_file_name)
